In [1]:
!pip install -U sacrebleu datasets transformers sentencepiece protobuf accelerate evaluate unbabel-comet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
from datasets import Dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import get_linear_schedule_with_warmup

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Load dataset with pandas
file_path = '/content/drive/MyDrive/combined_df.csv' 
df = pd.read_csv(file_path)

In [9]:
df

,Unnamed: 0,hr,nl
0,0,preuređenje,herinrichting
1,1,obradba,bewerking
2,2,prilagodba,aanpassing
3,3,notes,adresboekje
4,4,popis,adressenlijst
...,...,...,...
56081,56081,slikovit,beeldend
56082,56082,živahnost,levensechtheid
56083,56083,žmigati,knipperen
56084,56084,žrtvovati,ontzegging


In [10]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [11]:
df.dropna(inplace=True)

In [12]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,
                          Seq2SeqTrainingArguments, Seq2SeqTrainer)

# Split the DataFrame into training and validation DataFrames
train_df, val_df = train_test_split(df, test_size=0.05, random_state=42)

# Convert the training and validation DataFrames into Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Initialize tokenizer and model
model_checkpoint = "/content/drive/MyDrive/models-finetuning-1/mbart50_large_nl_hr/finetuned-mbart-large-50-nl-to-hr-mmt/checkpoint-46095"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Model configuration settings
model.config.max_length = 64
model.config.early_stopping = True
model.config.num_beams = 5

# Specify source and target languages
tokenizer.src_lang = "nl_XX"
tokenizer.tgt_lang = "hr_HR"

# Update preprocess function to reflect the change in source and target languages
def preprocess_function(examples):
    inputs = examples["nl"]  # Dutch is now the input
    targets = examples["hr"]  # Croatian is now the target
    model_inputs = tokenizer(inputs, max_length=64, truncation=True)

    # Tokenize the targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=64, truncation=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=['hr', 'nl', '__index_level_0__'])
val_dataset = val_dataset.map(preprocess_function, batched=True, remove_columns=['hr', 'nl', '__index_level_0__'])

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Update training arguments if necessary
args = Seq2SeqTrainingArguments(
    output_dir="finetuned-mbart-nl-to-hr",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-9,  
    per_device_train_batch_size=64,  
    per_device_eval_batch_size=64,
    weight_decay=0.1,
    num_train_epochs=3,
    predict_with_generate=True,  
    fp16=True,
    logging_dir='./logs',
    logging_steps=200,
    warmup_steps=0,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    label_smoothing_factor=0,
)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Ready to start training


Map:   0%|          | 0/53280 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2805 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.556300,2.497922
2,2.552700,2.496995
3,2.506400,2.496827


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception i

TrainOutput(global_step=2499, training_loss=2.5345181346464365, metrics={'train_runtime': 475.1341, 'train_samples_per_second': 336.41, 'train_steps_per_second': 5.26, 'total_flos': 4386880155549696.0, 'train_loss': 2.5345181346464365, 'epoch': 3.0})

In [14]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')
evaluation_df = pd.read_csv('/content/drive/MyDrive/evaluation_df.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNor

In [16]:
def translate_dutch_to_croatian(texts):
    tokenizer.src_lang = "nl_XX"  # Set source language to Dutch
    encoded_nl = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=64).to('cuda')
    generated_tokens = model.generate(**encoded_nl, forced_bos_token_id=tokenizer.lang_code_to_id["hr_HR"])
    return [tokenizer.decode(g, skip_special_tokens=True) for g in generated_tokens]

In [18]:
from tqdm.auto import tqdm
from sacrebleu.metrics import BLEU

bleu = BLEU()

# Prepare lists for translations and BLEU scores
dutch_to_croatian_translations = []
dutch_to_croatian_bleu_scores = []

batch_size = 64  # Set the batch size for processing
for i in tqdm(range(0, len(evaluation_df), batch_size)):
    batch = evaluation_df.iloc[i:i+batch_size]

    # Translate batches from Dutch to Croatian
    translated_croatian_batch = translate_dutch_to_croatian(batch['Dutch'].tolist())  # Batch translation function

    # Extend the translation list with the translated batch
    dutch_to_croatian_translations.extend(translated_croatian_batch)

    # Compute BLEU scores for Croatian translations in batch
    for translated_croatian, original_croatian in zip(translated_croatian_batch, batch['Croatian'].tolist()):
        croatian_score = bleu.corpus_score([translated_croatian], [[original_croatian]])
        dutch_to_croatian_bleu_scores.append(croatian_score.score)

# Update DataFrame with translated Croatian and BLEU scores
evaluation_df['Translated Croatian'] = dutch_to_croatian_translations
evaluation_df['Dutch to Croatian BLEU'] = dutch_to_croatian_bleu_scores

  0%|          | 0/35 [00:00<?, ?it/s]

In [19]:
evaluation_df

,Unnamed: 0,Dutch,Croatian,Translated Croatian,Dutch to Croatian BLEU
0,0,Op maandag kondigden wetenschappers van de Sta...,U ponedjeljak su znanstvenici s Medicinskog fa...,U ponedjeljak su znanstvenici iz Medicinskog f...,21.088685
1,1,Hoofdonderzoekers zeggen dat dit kan leiden to...,Voditelji istraživanja izjavili su da bi ovo o...,Glavni znanstvenici kažu da to može dovesti do...,14.456852
2,2,De JAS 39C Gripen stortte rond 09.30 uur lokal...,JAS 39C Gripen srušio se na pistu i eksplodira...,JAS 39C Gripen srušio se na piste oko 09 30 sa...,39.328881
3,3,De piloot werd geïdentificeerd als majoor Dilo...,Pilot je identificiran kao zapovjednik eskadri...,Pilot je identificiran kao majstor Dilokrit Pa...,44.124845
4,4,De lokale media meldt dat er tijdens een actie...,Lokalni mediji izvješćuju da je došlo do prevr...,Lokalna medija izvješćuju da je vatrogasna kol...,7.947829
...,...,...,...,...,...
2221,212,Mijn naam is Andrea.,Moje ime je Andrea.,Moje ime je Andrea.,100.000000
2222,213,Jupiter is een planeet.,Jupiter je planet.,Jupiter je planet.,100.000000
2223,214,Venus is een planeet.,Venera je planet.,Venera je planet.,100.000000
2224,215,God is een olifant.,Bog je slon.,Bog je slon.,100.000000


In [24]:
import pandas as pd
from nltk.translate.meteor_score import single_meteor_score
from nltk import word_tokenize
import nltk
from tqdm import tqdm

# Download NLTK's tokenizer models
nltk.download('punkt')
nltk.download('wordnet')

# Initialize an empty list for the METEOR scores
dutch_to_croatian_meteor_scores = []

# Loop through each row in the DataFrame
for _, row in tqdm(evaluation_df.iterrows(), total=evaluation_df.shape[0]):
    # Tokenize the Dutch and Croatian sentences
    original_croatian = word_tokenize(row['Croatian'])  # Original Dutch sentences
    translated_croatian = word_tokenize(row['Translated Croatian'])  # Translated sentences from Dutch to Croatian

    # Calculate the METEOR score for Dutch to Croatian translation
    meteor_score = single_meteor_score(original_croatian, translated_croatian)
    dutch_to_croatian_meteor_scores.append(meteor_score)

# Add the METEOR scores to DataFrame
evaluation_df['Dutch to Croatian METEOR'] = dutch_to_croatian_meteor_scores

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
100%|██████████| 2226/2226 [00:02<00:00, 976.60it/s] 


In [21]:
from evaluate import load
from tqdm import tqdm

# Load the COMET metric
comet_metric = load('comet')

# Function to compute COMET scores in batches
def compute_comet_in_batches(sources, predictions, references, batch_size=100):
    scores = []
    for i in tqdm(range(0, len(sources), batch_size)):
        batch_sources = sources[i:i + batch_size]
        batch_predictions = predictions[i:i + batch_size]
        batch_references = references[i:i + batch_size]
        batch_results = comet_metric.compute(predictions=batch_predictions, references=batch_references, sources=batch_sources)
        scores.extend(batch_results["scores"])
    return scores

# Dutch to Croatian COMET scores
data_dut_to_cro = {
    "sources": evaluation_df['Dutch'].tolist(),  # Original Dutch sentences
    "predictions": evaluation_df['Translated Croatian'].tolist(),  # Translated sentences in Croatian
    "references": evaluation_df['Croatian'].tolist()  # Reference sentences in Croatian
}

# Calculate the COMET scores
dut_to_cro_scores = compute_comet_in_batches(data_dut_to_cro['sources'], data_dut_to_cro['predictions'], data_dut_to_cro['references'])
evaluation_df['Dutch to Croatian COMET'] = [round(score, 5) for score in dut_to_cro_scores]

# Display the updated DataFrame
print(evaluation_df.head())

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.2.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:188: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
  0%|          | 0/23 [00:00<?, ?it/s]INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA d

   Unnamed: 0                                              Dutch  \
0           0  Op maandag kondigden wetenschappers van de Sta...   
1           1  Hoofdonderzoekers zeggen dat dit kan leiden to...   
2           2  De JAS 39C Gripen stortte rond 09.30 uur lokal...   
3           3  De piloot werd geïdentificeerd als majoor Dilo...   
4           4  De lokale media meldt dat er tijdens een actie...   

                                            Croatian  \
0  U ponedjeljak su znanstvenici s Medicinskog fa...   
1  Voditelji istraživanja izjavili su da bi ovo o...   
2  JAS 39C Gripen srušio se na pistu i eksplodira...   
3  Pilot je identificiran kao zapovjednik eskadri...   
4  Lokalni mediji izvješćuju da je došlo do prevr...   

                                 Translated Croatian  Dutch to Croatian BLEU  \
0  U ponedjeljak su znanstvenici iz Medicinskog f...               21.088685   
1  Glavni znanstvenici kažu da to može dovesti do...               14.456852   
2  JAS 39C Gri

In [25]:
# Calculate average scores for each metric
average_dutch_to_croatian_bleu = evaluation_df['Dutch to Croatian BLEU'].mean()
average_dutch_to_croatian_meteor = evaluation_df['Dutch to Croatian METEOR'].mean()
average_dutch_to_croatian_comet = evaluation_df['Dutch to Croatian COMET'].mean()

# Print the averages
print("Translation scores from Dutch to Croatian:")
print(f"Average BLEU: {average_dutch_to_croatian_bleu}")
print(f"Average METEOR: {average_dutch_to_croatian_meteor}")
print(f"Average COMET: {average_dutch_to_croatian_comet}")

Translation scores from Dutch to Croatian:
Average BLEU: 18.162807742709283
Average METEOR: 0.42636732702990593
Average COMET: 0.8518599371069183


In [26]:
evaluation_df.to_csv('/content/drive/MyDrive/evaluation_df_mbart50_nl_hr.csv')